In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

In [2]:
path = Path('../Data/')

## Import first database 

In [3]:
df = pd.read_csv(path / "user_data.csv")
df.head(2)

,user_id,country,age,gender,start_date
0,9541124,DK,22,male,2018-05-05
1,5946065,US,15,male,2018-01-24


In [4]:
df.shape

(600000, 5)

In [5]:
db = sqlite3.connect(path / "user_data.db")

In [7]:
c = db.cursor()
try:
    c.execute("""
            CREATE TABLE users (
                user_id int,
                country text,
                age int,
                gender text,
                start_date text
            )
        """)
except sqlite3.OperationalError:
    print('table already exists')

Export existing csv file to sql

In [8]:
df.to_sql('users', db, if_exists='replace', index=False)

Visualize the database in sql

In [9]:
c.execute("SELECT * FROM users")

print(len(c.fetchall()))

600000


In [10]:
c.execute("SELECT rowid, * FROM users")
items = c.fetchmany(3)
for item in items:
    print(item)

(1, 9541124, 'DK', 22, 'male', '2018-05-05')
(2, 5946065, 'US', 15, 'male', '2018-01-24')
(3, 4180986, 'US', 32, 'female', '2018-12-17')


It looks alright.

Now insert one value into the database

In [11]:
c.execute("""
    INSERT INTO users VALUES (9298372, 'US', 19, 'female', '2018-07-07')
    """)

db.commit()

print('Command executed correctly')

Command executed correctly


In [12]:
c.execute("SELECT rowid, * FROM users WHERE user_id = 9298372")
items = c.fetchall()

for item in items:
    print(item)

(600001, 9298372, 'US', 19, 'female', '2018-07-07')


So it worked.

and so now insert at once many values into the database:

In [13]:
many_ = [
    (9298372, 'US', 49, 'male', '2018-07-07'),
    (9298373, 'IT', 29, 'male', '2018-08-07'),
    (9298374, 'DK', 39, 'female', '2018-09-07'),
]

c.executemany("INSERT INTO users VALUES (?, ?, ?, ? ,?)", many_)

db.commit()

In [14]:
c.execute("SELECT rowid, * FROM users WHERE rowid > 600000")

items = c.fetchall()

for item in items:
    print(item)

(600001, 9298372, 'US', 19, 'female', '2018-07-07')
(600002, 9298372, 'US', 49, 'male', '2018-07-07')
(600003, 9298373, 'IT', 29, 'male', '2018-08-07')
(600004, 9298374, 'DK', 39, 'female', '2018-09-07')


In [15]:
c.execute("SELECT * FROM users")
print(len(c.fetchall()))

600004


Everything looks fine. 

Let's play around a bit

In [16]:
c.execute("SELECT * FROM users WHERE country = 'US' AND age < 25")

print(len(c.fetchall()))

138265


In [17]:
c.execute("""SELECT * FROM users 
        WHERE country = 'US' 
        AND age < 25
        AND gender = 'male'
    """)

print(len(c.fetchall()))

93246


In [18]:
c.execute("""SELECT * FROM users 
        WHERE country = 'US' 
        AND age BETWEEN 20 AND 29
        AND gender = 'male'
    """)

print(len(c.fetchall()))

77583


## Import another database into a second table

In [19]:
df_a = pd.read_csv(path / "prod_A.csv")
df_a.head(2)

,ds,userid
0,2020-01-01,3384044
1,2020-01-01,5543378


In [20]:
df_a.shape

(678802, 2)

In [21]:
df_a.rename(columns = {'userid': 'user_id'}, inplace=True)
df_a.head(2)

,ds,user_id
0,2020-01-01,3384044
1,2020-01-01,5543378


In [22]:
c = db.cursor()
try:
    c.execute("""
            CREATE TABLE users_a (
                ds text,
                user_id int
            )
        """)
except sqlite3.OperationalError:
    print('table already exists')
    
db.commit()

In [23]:
#c_a.execute("DROP TABLE users_a")
#db_a.commit()

In [24]:
df_a.to_sql('users_a', db, if_exists='replace', index=False)

In [25]:
c.execute("SELECT * FROM users_a")

print(len(c.fetchall()))

678802


In [26]:
c.execute("SELECT * FROM users_a ORDER BY ds LIMIT 2")

print(c.fetchall())

[('2020-01-01', 3384044), ('2020-01-01', 5543378)]


## LEFT JOIN
so that we have all the information we 
are provided with about the customers
who purchases product A

In [27]:
c.execute("""
        SELECT * FROM users_a
        LEFT JOIN users ON users.user_id = users_a.user_id
    """)

items = c.fetchmany(3)

for item in items:
    print(item)

('2020-01-01', 3384044, 3384044, 'US', 19, 'male', '2018-04-23')
('2020-01-01', 5543378, 5543378, 'US', 22, 'female', '2018-01-03')
('2020-01-01', 5773857, 5773857, 'US', 22, 'female', '2018-12-14')


In [28]:
c.execute("""
        SELECT * FROM users_a
        LEFT JOIN users ON users.user_id = users_a.user_id
    """)

print(len(c.fetchall()))

678802


## Import other database into a third table

In [29]:
df_b = pd.read_csv(path / "prod_B.csv")
df_b.head(2)

,ds,userid
0,2020-01-01,3384044
1,2020-01-01,5543378


In [30]:
df_b.shape

(233853, 2)

In [32]:
with db:
    c.execute("""CREATE TABLE users_b(
            ds text,
            userid int
            )
        """)

In [34]:
df_b.to_sql('users_b', db, if_exists='replace', index=False)

In [35]:
c.execute("""SELECT * FROM users_b
        WHERE ds BETWEEN "2020-01-01" AND "2020-01-02"
        ORDER BY ds DESC
        LIMIT 20
    """)

for item in c.fetchall():
    print(item)

('2020-01-02', 9786488)
('2020-01-02', 2594765)
('2020-01-02', 7752497)
('2020-01-02', 8471765)
('2020-01-02', 3654328)
('2020-01-02', 9714961)
('2020-01-02', 6572541)
('2020-01-02', 8912150)
('2020-01-02', 9779451)
('2020-01-02', 1869675)
('2020-01-02', 6045728)
('2020-01-02', 9039475)
('2020-01-02', 2781409)
('2020-01-02', 4328457)
('2020-01-02', 7253423)
('2020-01-02', 7950890)
('2020-01-02', 1635867)
('2020-01-02', 7295311)
('2020-01-02', 9167532)
('2020-01-02', 9526008)


## USE INNER JOIN
to find all the users who purchased both products A and B

In [36]:
c.execute("""SELECT * FROM users_a
        INNER JOIN users_b ON users_a.user_id = users_b.userid
    """)

print(len(c.fetchall()))

359576


## SELECT DISTINCT clause

This cannot be the number we are looking for, as the number of users who purchased B are 233853, so less than this. This is due to the fact that we are not considering **distinct** values of user_id's. To fix this, we need to use SELECT DISTINCT

In [37]:
c.execute("""SELECT DISTINCT a.user_id, b.userid
        FROM users_a AS a
        INNER JOIN users_b AS b
        ON a.user_id = b.userid
    """)

print(len(c.fetchall()))

items = c.fetchmany(50)

for item in items:
    print(item)

118887
